In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [36]:
import os
import re
import time
from typing import List, Any, Union
import pandas as pd
import multiprocessing as mp
import math
import copy as cp

In [31]:
def buildGraph(data):
    graph = {}
    for line in data:
        line = re.sub(r"(?<=\d)\n", "", line)
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        graph[line[0]] = line[1:]
    return graph

def reFormatTestData(data):
    format_data = []
    for line in data:
        line = re.sub(r"(?<=\d)\n", "", line)
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        line[0] = int(line[0])
        format_data.append(line)
    return format_data

def ReformatTrainData(data):
    format_data = []
    for line in data:
        line = re.sub(r"(?<=\d)\n", "", line)
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        line[0] = int(line[0])
        format_data.append(line)
    return format_data

# read train data
file_dir = 'comp90051-2020-sem2-proj1/'
file_name = "SmallTrain.txt"

global graph,train_set,test_set
with open(os.path.join(file_dir, file_name)) as f:
    graph = f.readlines()
    graph = buildGraph(graph)
    train_set = ReformatTrainData(graph)

# read test data
test_name = "SmallTest.txt"
with open(os.path.join(file_dir, test_name)) as f:
    f.readline()
    test_set = f.readlines()
    test_set = reFormatTestData(test_set)


In [22]:
#testing
print(len(graph))
print(len(train_set))
print(len(test_set))
#print(type(test_set))

3
3
2


Feature Extraction

In [37]:
'''
### Feature Extraction
'''
# Combin in and out
# input:str,dic
def getNeighbors(node, graph):
    follows = GetFollows(node, graph)
    fans = GetFans(node, graph)
    neighbors = follows.union(fans)
    return neighbors

def getNeighborsNumber(node, graph):
    follows = GetFollows(node, graph)
    fans = GetFans(node, graph)
    neighbors = follows.union(fans)
    return len(neighbors)

# out
def GetFollows(node, graph):
    follows = set()
    try:
        for item in graph[node]:
            follows.add(item)
        return follows
    except:
        return follows

# in
def GetFans(node, graph):
    fans = set()
    for key, value in graph.items():
        if node in value:
            fans.add(key)
    return fans

# Common Neighbors in Level 1
# numCommonCoauthor(x,y)
def commonNeighborsLevel1(a, b, graph):
    neighbors_a = getNeighbors(a, graph)
    neighbors_b = getNeighbors(b, graph)
    return len(neighbors_a.intersection(neighbors_b))

#Jaccard’s Coefficient
#两个人共同邻居的数量在他们所有好友数量中占比
def GetJaccardCoefficient(user_X,user_y,graph):
    neighbors_x = getNeighbors(user_X, graph)
    neighbors_y = getNeighbors(user_y, graph)
    CommonNeighbors=len(neighbors_x.intersection(neighbors_y))
    AllNeighborss=len(neighbors_x.union(neighbors_y))
    if AllNeighborss>=0:
        JaccardCoefficient=CommonNeighbors/AllNeighborss
    return JaccardCoefficient

#Adamic
#邻居的邻居数量越少，就越凸显它作为“中间人”的重要性，毕竟一共只认识那么少人，却恰好是x，y的好朋友

def GetAdamic(user_X,user_y,graph):
    neighbors_x = getNeighbors(user_X, graph)
    neighbors_y = getNeighbors(user_y, graph)
    CommonNeighbors=neighbors_x.intersection(neighbors_y)
    Adamic=0
    for commonNeighbor in CommonNeighbors:
        NeighborOfCommonNeighbor=getNeighborsNumber(commonNeighbor,graph)
        if NeighborOfCommonNeighbor!=0 and NeighborOfCommonNeighbor!=1:
            Adamic+=1/math.log(NeighborOfCommonNeighbor)

    return Adamic

#sumCoauthor


# Common Neighbors in Level 2
def commonNeighborsLevel2(a, b, graph):
    score = 0
    follows_a = GetFollows(a, graph)
    if follows_a:
        for node in follows_a:
            score += commonNeighborsLevel1(b, node, graph)
        return float(score / len(follows_a))
    else:
        return 0

debugging methods

In [38]:
'''
def getTestFeatures(record: list) -> list:
    node= [record[0],GetAdamic(record[1], record[-1], graph)]
    #print(node)
    return node
features=[]
for record in test_set:
    features.append(getTestFeatures(record))
'''

Feature Extraction


In [39]:
# Feature Extraction
def getTestFeatures(record: list) -> list:
    node= [record[0],len(GetFans(record[1],graph)),len(GetFollows(record[1],graph)),len(GetFans(record[-1],graph)),len(GetFollows(record[-1],graph)),commonNeighborsLevel1(record[1], record[-1], graph)]
    #print(node)
    return node
features=[]
for record in test_set:
    features.append(getTestFeatures(record))

#print(features)
title = ["Id", "InSource", "OutSource", "InSink", "OutSink", "CommonNeighboursLevel1"]
test_pd = pd.DataFrame(columns=title, data=features)
test_pd.to_csv('test_set_features.csv', encoding='utf-8')

test_set.clear()
features.clear()
#test_pd.clear()

following can be ignored.


In [35]:
'''
### Train Set Building
'''
def buildPositiveTrainSet(temp_graph):
    train_set_positive = set()
    # Positive train data
    for key in temp_graph:
        for each in graph[key]:
            train_set_positive.add((key,each,1)) #setting category of positive as 1 eg:(seed,followee1,1)
    return list(train_set_positive)

train_set_positive = buildPositiveTrainSet(graph)
print(train_set_positive)
print(len(train_set_positive))

[('4', '1', 1), ('4', '3', 1), ('4', '5', 1), ('2', '3', 1), ('1', '2', 1)]
5


In [ ]:
# Feature Extraction
def getTrainFeatures(record):
    node= [record[0],len(GetFans(record[1],graph)),len(GetFollows(record[1],graph)),len(GetFans(record[-1],graph)),len(GetFollows(record[-1],graph)),commonNeighborsLevel1(record[1], record[-1], graph),record[-1]]
    return node

print(train_set)




title = ["Id", "InSource", "OutSource", "InSink", "OutSink", "CommonNeighboursLevel1"]
test_pd = pd.DataFrame(columns=title, data=features)
test_pd.to_csv('./train_set_positive_features.csv', encoding='utf-8')

features.clear()
test_pd.clear()

In [ ]:
def buildNegativeTrainSet(graph):
    train_set_negative = set()
    # Positive train data
    train_set_negative = set()
    for key in graph:
        for each in graph[key]:
            if((each,key) not in train_set_positive):
                train_set_negative.add((each,key,0))

    keys = list(graph.keys())
    for i in range(len(keys)-1):
        if ((keys[i],keys[i+1]) not in train_set_positive):
            train_set_negative.add((keys[i],keys[i+1]))
        if ((keys[i+1],keys[i]) not in train_set_positive):
            train_set_negative.add((keys[i+1],keys[i]))
    return list(train_set_negative)
#print(len(train_set_negative))
train_set_positive.clear()

In [ ]:
# Feature Extraction
def getTrainFeatures(record):
    node= [record[0],len(GetFans(record[1],graph)),len(GetFollows(record[1],graph)),len(GetFans(record[-1],graph)),len(GetFollows(record[-1],graph)),commonNeighborsLevel1(record[1], record[-1], graph),record[-1]]
    return node

pool = mp.Pool(mp.cpu_count())
features = pool.map(getTrainFeatures, train_set)
pool.close()
pool.join()

title = ["Id", "InA", "OutA", "InB", "OutB", "CommonNeighboursLevel1", "Label"]
test_pd = pd.DataFrame(columns=title, data=features)
test_pd.to_csv('./train_set_negatitive_features.csv', encoding='utf-8')